# Epidemic at a university

This app will simulate an epidemic on a university campus (or more broadly, a fairly small, 'closed' population). Key assumptions of the model are:

* There is a simple Susceptible -> Exposed -> Infectious -> Recovered framework
* The exposed and infectious period both have mean length of 1 week, and immunity is long-lasting.
* 1000 people live in households of 10. Most mixing is within the household, with a small amount of random mixing.
* Everyone attends a daily 2-hour class with a fixed group of 100 students.

The app will show the outcome in the default case when no interventions are applied. You can use the sliders below to choose a mix of different interventions:

* **Testing** - There is weekly testing of a percentage of the population with immediate results. The tests are 95% accurate at finding cases (exposed and infectious) and have a 1% false positive rate for susceptible and recovered individuals.
* **Isolation** - Positive cases and those contact traced (below) fully isolate for a chosen number of days.
* **Tracing** - A chosen (maximum) number of close contacts are identified from the household and class of identified cases to quarantine for the isolation period.
* **Mixing reduction** - Restrictions on mixing of individuals outside of households and classes can be enacted.
* **Class size** - The size of the daily classes can be lowered (creating more classes).

Once you have chosen your intervention, click 'Run Interact'. The model will run 10 times (each run can take 10-20 seconds) and then plot the output both with the default assumptions and your chosen interventions. It will also tell you the peak and total number infected.

In [5]:
### Import the necessary libraries
#!jupyter nbextension enable --py widgetsnbextension --sys-prefix
#!jupyter serverextension enable voila --sys-prefix
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import ipywidgets as widgets
from ipywidgets import interact_manual

style = {'description_width': 'initial'}
test_w=widgets.IntSlider(min=0, max=100, step=5, value=0, description='% Testing',style=style)
isolate_w=widgets.IntSlider(min=1, max=10, step=1, value=0, description='Days isolating',style=style)
trace_w=widgets.IntSlider(min=0, max=10, step=1, value=0, description='Max no. contacts',style=style)
class_w=widgets.IntSlider(min=10, max=100, step=10, value=100, description='Class size',style=style)
mix_w=widgets.IntSlider(min=0, max=100, step=10, value=0, description='% Mixing reduction',style=style)

### Global constants
def runmodel():
    
    POPSIZE=1000 # Total individuals in population
    NOHOUSES=100 # No. of households (mean household size = POPSIZE / NOHOUSES)

    OMEGA=1/7 # Latent rate
    GAMMA=1/7 # Recovery rate
    WANE=0 # Waning immunity rate
    ISOLATE=5
    BETA=np.zeros(2)
    BETA[0]=0.02 # Transmission coefficient at home
    BETA[1]=0.02 # Transmission coefficient at class
    #BETASS=0.02

    REPS=10 # No. of replicate runs
    I0=10 # Initial no. of infected individuals (mean)
    MAX_TIME=150 # Max time
    CLASS_START = 0.4 # Time in day when move to class 
    CLASS_END = 0.48333 # Time in day when move to house
    MIXD=0.02
    TRACINGD=0
    TESTNOD=0
    CLASSD=100

    statstore=np.zeros(4)

    plt.rcParams.update({'font.size': 16})
    
    solstore=np.zeros((10,150))
    solstore[0]=[0.00, 10.00, 12.00, 12.00, 13.00, 14.00, 21.00, 27.00, 32.00, 39.00, 44.00, 46.00, 56.00, 72.00, 79.00, 88.00, 102.00, 117.00, 129.00, 153.00, 155.00, 172.00, 178.00, 184.00, 188.00, 199.00, 211.00, 217.00, 218.00, 217.00, 209.00, 211.00, 209.00, 206.00, 199.00, 197.00, 196.00, 189.00, 173.00, 164.00, 153.00, 144.00, 136.00, 125.00, 122.00, 113.00, 99.00, 103.00, 93.00, 85.00, 80.00, 74.00, 66.00, 65.00, 60.00, 57.00, 53.00, 48.00, 40.00, 36.00, 33.00, 31.00, 26.00, 26.00, 23.00, 23.00, 22.00, 21.00, 18.00, 15.00, 13.00, 11.00, 9.00, 8.00, 8.00, 8.00, 7.00, 8.00, 9.00, 10.00, 10.00, 9.00, 9.00, 9.00, 6.00, 5.00, 5.00, 5.00, 4.00, 4.00, 3.00, 3.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[1]=[0.00, 10.00, 13.00, 16.00, 15.00, 18.00, 22.00, 26.00, 25.00, 31.00, 37.00, 43.00, 57.00, 68.00, 72.00, 80.00, 83.00, 107.00, 117.00, 130.00, 141.00, 153.00, 169.00, 189.00, 193.00, 212.00, 227.00, 233.00, 234.00, 238.00, 233.00, 218.00, 225.00, 211.00, 199.00, 187.00, 199.00, 197.00, 185.00, 173.00, 170.00, 151.00, 138.00, 131.00, 121.00, 111.00, 105.00, 106.00, 98.00, 92.00, 84.00, 76.00, 71.00, 71.00, 61.00, 56.00, 53.00, 49.00, 44.00, 39.00, 36.00, 33.00, 33.00, 29.00, 24.00, 20.00, 18.00, 19.00, 16.00, 15.00, 14.00, 11.00, 10.00, 10.00, 10.00, 8.00, 7.00, 6.00, 6.00, 5.00, 5.00, 4.00, 1.00, 1.00, 2.00, 2.00, 2.00, 1.00, 2.00, 2.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[2]=[0.00, 10.00, 11.00, 10.00, 11.00, 12.00, 13.00, 21.00, 25.00, 27.00, 30.00, 33.00, 40.00, 49.00, 64.00, 67.00, 75.00, 88.00, 96.00, 110.00, 125.00, 140.00, 154.00, 149.00, 167.00, 170.00, 184.00, 203.00, 212.00, 224.00, 230.00, 225.00, 223.00, 221.00, 217.00, 214.00, 210.00, 198.00, 197.00, 183.00, 178.00, 171.00, 160.00, 150.00, 140.00, 126.00, 115.00, 106.00, 97.00, 86.00, 79.00, 71.00, 69.00, 70.00, 63.00, 60.00, 53.00, 46.00, 43.00, 40.00, 38.00, 35.00, 29.00, 28.00, 26.00, 27.00, 25.00, 22.00, 20.00, 18.00, 16.00, 14.00, 14.00, 12.00, 13.00, 12.00, 12.00, 9.00, 8.00, 8.00, 8.00, 9.00, 8.00, 7.00, 7.00, 7.00, 6.00, 5.00, 5.00, 5.00, 5.00, 5.00, 5.00, 3.00, 3.00, 3.00, 3.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[3]=[0.00, 9.00, 8.00, 9.00, 12.00, 14.00, 17.00, 24.00, 24.00, 25.00, 32.00, 36.00, 43.00, 54.00, 58.00, 78.00, 80.00, 87.00, 100.00, 103.00, 118.00, 142.00, 149.00, 160.00, 164.00, 164.00, 175.00, 185.00, 200.00, 207.00, 208.00, 202.00, 208.00, 201.00, 198.00, 190.00, 190.00, 186.00, 178.00, 172.00, 164.00, 158.00, 145.00, 138.00, 133.00, 122.00, 103.00, 93.00, 88.00, 83.00, 78.00, 65.00, 59.00, 54.00, 53.00, 54.00, 50.00, 48.00, 45.00, 43.00, 41.00, 36.00, 34.00, 30.00, 26.00, 22.00, 20.00, 20.00, 18.00, 18.00, 17.00, 16.00, 15.00, 15.00, 14.00, 12.00, 12.00, 10.00, 10.00, 10.00, 9.00, 8.00, 6.00, 5.00, 5.00, 5.00, 5.00, 4.00, 5.00, 5.00, 5.00, 5.00, 4.00, 4.00, 4.00, 4.00, 4.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[4]=[0.00, 11.00, 11.00, 12.00, 16.00, 18.00, 17.00, 20.00, 19.00, 21.00, 28.00, 36.00, 45.00, 55.00, 73.00, 75.00, 90.00, 99.00, 106.00, 127.00, 133.00, 149.00, 169.00, 170.00, 188.00, 199.00, 208.00, 210.00, 212.00, 219.00, 213.00, 231.00, 230.00, 220.00, 211.00, 201.00, 183.00, 184.00, 176.00, 168.00, 160.00, 154.00, 142.00, 131.00, 132.00, 120.00, 112.00, 106.00, 103.00, 95.00, 87.00, 73.00, 64.00, 58.00, 55.00, 55.00, 48.00, 44.00, 40.00, 42.00, 38.00, 39.00, 38.00, 35.00, 31.00, 28.00, 28.00, 22.00, 18.00, 17.00, 17.00, 12.00, 13.00, 12.00, 10.00, 10.00, 11.00, 10.00, 9.00, 7.00, 7.00, 7.00, 7.00, 7.00, 7.00, 6.00, 7.00, 7.00, 7.00, 7.00, 6.00, 6.00, 6.00, 6.00, 5.00, 5.00, 4.00, 4.00, 4.00, 4.00, 4.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[5]=[0.00, 10.00, 9.00, 9.00, 13.00, 11.00, 16.00, 19.00, 17.00, 14.00, 14.00, 17.00, 26.00, 34.00, 42.00, 50.00, 57.00, 65.00, 72.00, 74.00, 87.00, 101.00, 108.00, 124.00, 144.00, 162.00, 177.00, 184.00, 197.00, 206.00, 220.00, 229.00, 233.00, 247.00, 245.00, 239.00, 222.00, 222.00, 216.00, 209.00, 211.00, 203.00, 184.00, 185.00, 166.00, 153.00, 139.00, 135.00, 125.00, 106.00, 96.00, 88.00, 83.00, 73.00, 66.00, 60.00, 54.00, 54.00, 50.00, 45.00, 43.00, 33.00, 27.00, 26.00, 20.00, 18.00, 15.00, 11.00, 9.00, 5.00, 3.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[6]=[0.00, 11.00, 8.00, 8.00, 8.00, 12.00, 15.00, 21.00, 20.00, 23.00, 27.00, 29.00, 37.00, 35.00, 46.00, 54.00, 59.00, 78.00, 86.00, 94.00, 98.00, 110.00, 125.00, 138.00, 145.00, 148.00, 152.00, 160.00, 167.00, 179.00, 197.00, 200.00, 209.00, 209.00, 210.00, 213.00, 206.00, 204.00, 197.00, 189.00, 184.00, 173.00, 178.00, 166.00, 155.00, 144.00, 140.00, 131.00, 121.00, 120.00, 109.00, 111.00, 102.00, 94.00, 86.00, 81.00, 77.00, 73.00, 67.00, 61.00, 58.00, 49.00, 47.00, 43.00, 40.00, 37.00, 34.00, 33.00, 27.00, 25.00, 25.00, 21.00, 21.00, 19.00, 17.00, 14.00, 11.00, 10.00, 10.00, 8.00, 7.00, 6.00, 6.00, 5.00, 5.00, 5.00, 5.00, 5.00, 3.00, 3.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[7]=[0.00, 9.00, 14.00, 13.00, 12.00, 12.00, 15.00, 16.00, 18.00, 19.00, 23.00, 27.00, 34.00, 43.00, 53.00, 66.00, 73.00, 79.00, 74.00, 88.00, 109.00, 119.00, 133.00, 153.00, 166.00, 171.00, 183.00, 184.00, 211.00, 238.00, 242.00, 255.00, 247.00, 247.00, 247.00, 236.00, 227.00, 209.00, 193.00, 186.00, 169.00, 160.00, 150.00, 142.00, 128.00, 114.00, 111.00, 101.00, 94.00, 81.00, 83.00, 76.00, 72.00, 66.00, 61.00, 54.00, 48.00, 45.00, 43.00, 41.00, 40.00, 37.00, 36.00, 31.00, 28.00, 26.00, 22.00, 23.00, 20.00, 16.00, 15.00, 14.00, 13.00, 12.00, 11.00, 11.00, 10.00, 9.00, 8.00, 7.00, 7.00, 8.00, 9.00, 8.00, 8.00, 7.00, 8.00, 5.00, 3.00, 2.00, 2.00, 2.00, 2.00, 2.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[8]=[0.00, 11.00, 12.00, 14.00, 15.00, 15.00, 20.00, 27.00, 28.00, 30.00, 40.00, 51.00, 49.00, 52.00, 74.00, 86.00, 96.00, 110.00, 122.00, 135.00, 140.00, 152.00, 169.00, 187.00, 194.00, 207.00, 211.00, 230.00, 231.00, 236.00, 230.00, 234.00, 249.00, 246.00, 248.00, 240.00, 223.00, 208.00, 194.00, 186.00, 177.00, 167.00, 167.00, 156.00, 155.00, 150.00, 140.00, 120.00, 112.00, 100.00, 89.00, 83.00, 77.00, 66.00, 56.00, 55.00, 52.00, 46.00, 38.00, 34.00, 32.00, 25.00, 24.00, 22.00, 19.00, 19.00, 19.00, 17.00, 14.00, 13.00, 10.00, 9.00, 8.00, 6.00, 5.00, 5.00, 3.00, 3.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]
    solstore[9]=[0.00, 11.00, 10.00, 9.00, 14.00, 15.00, 16.00, 22.00, 30.00, 36.00, 45.00, 53.00, 56.00, 65.00, 74.00, 94.00, 107.00, 124.00, 145.00, 164.00, 164.00, 185.00, 209.00, 210.00, 226.00, 236.00, 249.00, 250.00, 247.00, 247.00, 242.00, 235.00, 230.00, 217.00, 210.00, 198.00, 192.00, 181.00, 178.00, 164.00, 146.00, 135.00, 119.00, 107.00, 99.00, 93.00, 82.00, 78.00, 69.00, 68.00, 63.00, 58.00, 53.00, 47.00, 42.00, 40.00, 36.00, 31.00, 31.00, 26.00, 23.00, 21.00, 20.00, 18.00, 16.00, 15.00, 14.00, 13.00, 12.00, 13.00, 13.00, 11.00, 10.00, 9.00, 9.00, 9.00, 8.00, 5.00, 5.00, 3.00, 3.00, 3.00, 3.00, 2.00, 2.00, 1.00, 1.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00, 0.00]

    ### Functions

    # Function to count numbers in each compartment
    def count_type(type):
        return np.count_nonzero(PERSON[:,3] == type)

    # Function to check the current scale
    def findscale(home):
        # Check location
        loc = NOHOUSES if home==1 else NOCLASSES
        counts = np.zeros((loc, 4))
        icounts = np.zeros((loc, 4))
        icountss = np.zeros((loc,4))
        for j in range(0, loc):
            for c in range(1, 4):
                counts[j, c] = np.count_nonzero(np.logical_and(PERSON[:,home]==j, PERSON[:,3]==c)) 
            for c in [0,2]:
                icounts[j, c] = np.count_nonzero(np.logical_and(np.logical_and(PERSON[:,home]==j, PERSON[:,3]==c), PERSON[:,4]==0))
               # icountss[j, c] = np.count_nonzero(np.logical_and(np.logical_and(PERSON[0:200,home]==j, PERSON[0:200,3]), PERSON[0:200,4]==0))
        # Sum of all rates
        scale = np.sum(OMEGA*counts[:,1] + GAMMA*counts[:,2] + WANE*counts[:,3]) + (1-MIX)*BETA[home-1]*np.sum(icounts[:,0]*icounts[:,2])+MIX*(BETA[home-1]*np.sum(icounts[:,2]))*np.sum(icounts[:,0])
        return scale

    # Main function

    MIX=0.0002*(100-mix_w.value)
    TRACING=(trace_w.value)
    TESTNO=(10*test_w.value)  
    CLASS=(class_w.value)
    ISOLATE=(isolate_w.value)

    classes=CLASS
    NOCLASSES=int(np.ceil(POPSIZE/(classes))) 

    peaks=[] # For storing peak no. of infecteds
    tots=[] # For storing total no. infected

    for reps in range(0,REPS):
        
        if reps==0:
            print("Running simulation 1", end=" ")
        else:
            print(", %i" %int(reps+1), end=" ")

        shuff1=np.arange(POPSIZE)
        shuff2=np.arange(POPSIZE)

        np.random.shuffle(shuff1)
        np.random.shuffle(shuff2)

        PERSON=np.zeros((POPSIZE,5))
        # 0th col: ID - First 200 are superspreaders
        # 1st col: house
        # 2nd col: class
        # 3rd col: SIR status
        # 4th col: Isolation status
        infs=np.sort(np.random.choice(POPSIZE,I0))
        for i in range(0,POPSIZE):
            PERSON[i][0] = i
            if i in infs:
                PERSON[i][3] = 2 # Initially everyone susceptible except I0 individuals

        for index in range(NOHOUSES):
            for i in shuff1[index::NOHOUSES]:
                PERSON[i][1]=index

        for index in range(NOCLASSES):
            for i in shuff2[index::NOCLASSES]:
                PERSON[i][2]=index 

        # Some local constants / lists
        tsteps=[0]
        infecteds=[count_type(2)]
        susceptibles=[count_type(0)]
        exposed=[count_type(1)]
        current_t=0  
        home=2 # Everyone starts at home
        total_infections=0 # Since immunity wanes, count every infection event

        # Main run
        while current_t < MAX_TIME and np.any(PERSON[:,3] == 2):

            # Find proposed time to next event
            scale = findscale(home)
            dt = -np.log(np.random.uniform()) / scale
            proposed_t = tsteps[-1] + dt

            if int(proposed_t) > int(current_t): # Has to come first or misses days
                # If new day, change isolation status and test, then start again
                # Update isolation status
                for ppl in range(POPSIZE):
                    if PERSON[ppl,4]>=1:
                        PERSON[ppl,4]-=1

                # Jump is only one new day
                current_t = int(current_t)+1

                # Weekly Test
                if int(current_t)%7==1:

                    # Only non-isolators test
                    findx=np.where(PERSON[:,4]==0)
                    # Adjust test number if too many isolating
                    testnum = TESTNO if len(findx[0])>TESTNO else len(findx[0])
                    # Randomise list
                    np.random.shuffle(findx[0])
                    # Isolate Exposed and Infected individuals for 10 days
                    for tests in range(testnum):
                        if PERSON[findx[0][tests],3]==1 or PERSON[findx[0][tests],3]==2:
                            # False negative / ignore rate of 5%
                            if np.random.uniform()<0.95:
                                PERSON[findx[0][tests],4]=ISOLATE
                                # contact trace
                                num_traced = np.random.randint(TRACING+1)
                                case_contacts=np.where((PERSON[:,1]==PERSON[findx[0][tests],1]) | (PERSON[:,2]==PERSON[findx[0][tests],2]))
                                np.random.shuffle(case_contacts[0])
                                #print(num_traced,case_contacts[0].size)
                                for trace in range(num_traced):
                                    if PERSON[case_contacts[0][trace],4]==0:
                                        PERSON[case_contacts[0][trace],4]=ISOLATE
                        else:
                            # False positive rate of 1%
                            if np.random.uniform()>0.99:
                                PERSON[findx[0][tests],4]=ISOLATE
                                # contact trace
                                num_traced = np.random.randint(TRACING+1)
                                case_contacts=np.where((PERSON[:,1]==PERSON[findx[0][tests],1]) | (PERSON[:,2]==PERSON[findx[0][tests],2]))
                                np.random.shuffle(case_contacts[0])
                                for trace in range(num_traced):
                                    if PERSON[case_contacts[0][trace],4]==0:
                                        PERSON[case_contacts[0][trace],4]=ISOLATE
            elif home == 1 and proposed_t > int(proposed_t) + CLASS_START and proposed_t < int(proposed_t) + CLASS_END:
                # If students are home and proposed time of next event is later 
                # than class starts, then no event occurs before class starts
                current_t = int(proposed_t)+CLASS_START
                home = 2
            elif home == 2 and proposed_t > int(proposed_t) + CLASS_END:
                # If students are in class and proposed time of next event is 
                # later than class ends, then no event occurs before class ends
                current_t = int(proposed_t)+CLASS_END
                home = 1 
            else:
                # Next event occurs before class starts/ends
                current_t = proposed_t

                # Find event
                eventcheck = np.random.uniform()
                # Need to find non-isolating infecteds, superspreaders and susceptibles
                #countSS=np.count_nonzero(np.logical_and(PERSON[:,3]==2, PERSON[:,4]==0)) 
                countNSS=np.count_nonzero(np.logical_and(PERSON[:,3]==2, PERSON[:,4]==0)) 
                countsus=np.count_nonzero(np.logical_and(PERSON[:,3]==0, PERSON[:,4]==0))

                if eventcheck < GAMMA*infecteds[-1]/scale: #Event is recovery  

                    # If there are any infected people, randomly choose one to recover
                    infected_indices = np.where(PERSON[:,3] == 2)
                    if infected_indices:
                        PERSON[np.random.choice(infected_indices[0]), 3] = 3

                elif eventcheck < (GAMMA*infecteds[-1] + OMEGA*exposed[-1])/scale: # Event is latent->infected

                    # If there are any latents, randomly choose one to become infected
                    latent_indices = np.where(PERSON[:,3] == 1)
                    if latent_indices:
                        PERSON[np.random.choice(latent_indices[0]), 3] = 2                                                                                

                elif eventcheck < (GAMMA*infecteds[-1] + OMEGA*exposed[-1]+WANE*count_type(3))/scale: 
                    # Event is waned immunity
                    # If there are any immunes, randomly choose one to become susceptible
                    immune_indices = np.where(PERSON[:,3] == 3)
                    if immune_indices:
                        PERSON[np.random.choice(immune_indices[0]), 3] = 0                                 

                elif eventcheck <(GAMMA*infecteds[-1] + OMEGA*exposed[-1]+WANE*count_type(3) + MIX*(BETA[home-1]*countNSS)*countsus)/scale:

                    # Event is mixed transmission

                    sus_indices = np.where((PERSON[:,3] == 0) & (PERSON[:,4]==0))
                    if sus_indices[0].size>0:
                        PERSON[np.random.choice(sus_indices[0]), 3] = 1
                        total_infections+=1

                else: #Event is Transmission by househould contact
                    findx=np.where((PERSON[:,3]==0) & (PERSON[:,4]==0)) # Find susceptible hosts who are not isolating
                    np.random.shuffle(findx[0]) # randomly shuffle
                    for tryx in findx[0]:
                        loc=PERSON[tryx,home]
                        contacts=np.where(np.logical_and(np.logical_and(PERSON[:,home]==loc, PERSON[:,3]==2), PERSON[:,4]==0))
                        #contacts=np.where(PERSON[:,home]==loc)

                        if contacts[0].size>0:
                            PERSON[tryx,3]=1
                            total_infections+=1
                            break  

            # Update lists
            tsteps.append(current_t)
            infecteds.append(count_type(2))
            exposed.append(count_type(1))
            susceptibles.append(count_type(0))

            # Stop if infections has finished
            if infecteds[-1]==0 & exposed[-1]==0:
                tsteps.append(MAX_TIME)
                infecteds.append(0)
                exposed.append(0)
                susceptibles.append(count_type(0))
                break

       # print(count_type(0), count_type(1), count_type(2), count_type(3))

        # Find peak no. infected
        peaks.append(max(infecteds)/POPSIZE)
        tots.append(total_infections+I0)
        infplot=[]
        for i in range(len(infecteds)):
            infplot.append(infecteds[i]/10)
        if reps==0:
            plt.plot(tsteps,infplot,'b:',label='Interventions')
        else:
            plt.plot(tsteps,infplot,'b:')
    plt.plot(solstore[0]/10,'r:',label='Default')
    for i in range(1,10):
        plt.plot(solstore[i]/10,'r:')
    plt.xlabel('Days')
    plt.ylabel('% Infected')
        
    print("\nThe mean peak infected was reduced from 24% to", int(np.mean(peaks)*100), "%.")
    print("The mean total infected was reduced from 99% to", int(np.mean(tots)/10), "%.")

display(test_w)
display(isolate_w)
display(trace_w)
display(mix_w)
display(class_w)
interact_manual(runmodel)

IntSlider(value=0, description='% Testing', step=5, style=SliderStyle(description_width='initial'))

IntSlider(value=1, description='Days isolating', max=10, min=1, style=SliderStyle(description_width='initial'))

IntSlider(value=0, description='Max no. contacts', max=10, style=SliderStyle(description_width='initial'))

IntSlider(value=0, description='% Mixing reduction', step=10, style=SliderStyle(description_width='initial'))

IntSlider(value=100, description='Class size', min=10, step=10, style=SliderStyle(description_width='initial'))

interactive(children=(Button(description='Run Interact', style=ButtonStyle()), Output()), _dom_classes=('widget-interact',))

<function __main__.runmodel>

## Caveats

There are of course many important elements of a real epidemic that are not included in the model. These include:

* Vaccination - Everyone in this model has the same vaccine status (which could be assumed to be no-one or everyone vaccinated). A more accurate model would account for variation in vaccine coverage.
* Super-spreading - The model assumes everyone has the same potential to infect others. In many diseases 'super-spreading' plays a key role, where some individuals cause far more infections than others.
* External contacts - The model assumes that the population is compltely closed, with no infection arriving in from other sources.
* Testing accuracy - Testing accuracy in the model depends simply on your epidemic status, but a more realistic model would link this to infection growth dynamics in the body.

There are many more simplifications and assumptions besides. 

###### 